In [ ]:
pip install pandas requests selenium webdriver_manager beautifulsoup4 textblob

### **CODE GỐC**

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import re

def is_vietnamese(text):
    # Kiểm tra xem văn bản ký tự tiếng Việt không
    vietnamese_pattern = r'[àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]'
    return bool(re.search(vietnamese_pattern, text.lower()))

def scroll_to_element(driver, element):
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()

def get_reviews_for_rating(driver):
    try:
        review_section = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="reviewSectionComments"]'))
        )
        
        reviews = []
        i = 0
        while True:
            try:
                review_xpath = f'//*[@id="reviewSectionComments"]//*[@id="review-{i}"]/div[2]/div[1]/div[1]'
                review_element = review_section.find_element(By.XPATH, review_xpath)
                review_text = review_element.text.strip()
                
                if review_text:
                    reviews.append(review_text)
                
            except Exception as inner_e:
                if "no such element" in str(inner_e).lower():
                  break
                else:
                  print(f"Error retrieving review at index {i}: {inner_e}")
                  break
            i += 1
                
        return reviews
    except Exception as e:
        print(f"Error getting reviews: {str(e)}")
        return []

def label_reviews(rating_category, reviews):
    labels = {
        "9+ Hiếm Có": "positive",
        "8-9 Xuất Sắc": "positive",
        "7-8 Rất Tốt": "positive",
        "6-7 Tốt": "negative",
        "<6 Dưới Mức Mong Đợi": "negative"
    }
    
    labeled_reviews = []
    for review in reviews:
        if review:
            labeled_reviews.append({
                "review": review,
                "label": labels[rating_category]
            })
    return labeled_reviews

def get_hotel_name(driver):
    try:
        hotel_name_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="property-main-content"]/div[1]/div[2]/div[1]/h1'))
        )
        return hotel_name_element.text.strip()
    except Exception as e:
        print(f"Error getting hotel name: {str(e)}")
        return ""

def go_to_next_page(driver, current_page):
    try:
        next_page_xpath = f'//*[@id="reviews-panel-1"]/div[2]/div/span[2]/span[{current_page + 1}]'
        next_page_element = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, next_page_xpath))
        )
        scroll_to_element(driver, next_page_element)
        driver.execute_script("arguments[0].click();", next_page_element)
        time.sleep(2)  # Wait for the next page to load
        return True
    except (NoSuchElementException, TimeoutException):
        print("No next page found.")
        return False

def main():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-notifications')
    options.add_argument('--start-maximized')
    driver = webdriver.Chrome(options=options)

    try:
        # Gắn link của trang đánh giá khách sạn
        driver.get("https://www.agoda.com/vi-vn/ha-na-hotel-da-lat/hotel/dalat-vn.html")
        time.sleep(3)

        # Lấy tên khách sạn
        hotel_name = get_hotel_name(driver)
        print(f"Tên khách sạn: {hotel_name}")

        review_section = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="reviewSectionComments"]'))
        )
        scroll_to_element(driver, review_section)
        driver.execute_script("arguments[0].click();", review_section)
        time.sleep(2)

        # Chọn ngôn ngữ tiếng Việt
        language_select_xpath = '//*[@id="reviewFilterSection"]/div[1]/div[3]/div/div/label/div[1]/span/select'
        language_select = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, language_select_xpath))
        )
        language_select = Select(language_select)
        language_select.select_by_visible_text("Tiếng Việt")
        time.sleep(2)

        rating_categories = {
            "9+ Hiếm Có": '//*[@id="reviews-panel-1"]/div[1]/div[1]/div/div[3]/div/div/div/div/div[1]/span',
            "8-9 Xuất Sắc": '//*[@id="reviews-panel-1"]/div[1]/div[1]/div/div[3]/div/div/div/div/div[2]/span',
            "7-8 Rất Tốt": '//*[@id="reviews-panel-1"]/div[1]/div[1]/div/div[3]/div/div/div/div/div[3]/span',
            "6-7 Tốt": '//*[@id="reviews-panel-1"]/div[1]/div[1]/div/div[3]/div/div/div/div/div[4]/span',
            "<6 Dưới Mức Mong Đợi": '//*[@id="reviews-panel-1"]/div[1]/div[1]/div/div[3]/div/div/div/div/div[5]/span'
        }

        data = []
        for category, xpath in rating_categories.items():
            rating = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, xpath))
            )
            scroll_to_element(driver, rating)
            driver.execute_script("arguments[0].click();", rating)
            time.sleep(2)

            current_page = 1
            while True:
                print(f"\n=== Reviews for {category} (Page {current_page}) ===")
                reviews = get_reviews_for_rating(driver)
                labeled_reviews = label_reviews(category, reviews)

                for i, review in enumerate(labeled_reviews, 1):
                    print(f"\nReview {i}:")
                    print(f"Text: {review['review']}")
                    print(f"Label: {review['label']}")

                for review in labeled_reviews:
                    data.append({
                        "Tên Khách Sạn": hotel_name,
                        "Review": review['review'],
                        "Label": review['label']
                    })

                if go_to_next_page(driver, current_page):
                    current_page += 1
                else:
                    break

            driver.execute_script("arguments[0].click();", rating)
            time.sleep(1)

        # Lưu dữ liệu vào file Excel
        df = pd.DataFrame(data)
        df.to_excel("C:/Users/Admin/Downloads/FinalReview.xlsx", index=False)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

Tên khách sạn: Nhà trọ – John Cà phê và Bia (Hostel-John Cafe and Beer)

=== Reviews for 9+ Hiếm Có ===

Review 1:
Text: 10 điểm không có nhưng”
- Là 1 người sg chính hiệu nhưng lại muốn trở thành người đà lạt sau 2 ngày ở đây. - Phòng tập thể tuy hơi bất tiện nhưng lại sạch sẽ, ấm cúng, đầy đủ tiện nghi, có ngăn để đồ riêng, có ban công ở phòng rất chill. - Có khu sinh hoạt chung ở sân thượng, có rooftop để ngồi uống cf ngắm bình minh, hoàng hôn. - Đi du lịch 1 mình tiết kiệm thì còn nơi nào xứng đáng như vậy nữa 🫶
Label: positive

Review 2:
Text: CHỖ NÀY 10 ĐIỂM KHÔNG NÓI NHIỀU”
Mình thề với tầm giá tiền như thế này không thể ở một chỗ tốt hơn được. Phòng xịn, sạch sẽ, ấm áp, mình có căn nên đi du lịch ở mấy chỗ không phong thuỷ là mất ngủ, bóng đè, nhưng ngủ ở John giường như tẩm đá, ngủ ngon lắm. Chỗ ở trung tâm, mấy hàng ăn ngon toàn ở quanh phường này. Highly recommend!
Label: positive

Review 3:
Text: Dễ thương ấm cúng hợp cho chuyến xả hơi nhẹ nhàng”
Chỗ nghỉ nhỏ nhắn, ấm cúng,

### **CODE TEST**

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import re

def is_vietnamese(text):
    # Kiểm tra xem văn bản ký tự tiếng Việt không
    vietnamese_pattern = r'[àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]'
    return bool(re.search(vietnamese_pattern, text.lower()))

def scroll_to_element(driver, element):
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()

def get_reviews_for_rating(driver):
    try:
        review_section = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="reviewSectionComments"]'))
        )
        
        reviews = []
        i = 0
        while True:
            try:
                review_xpath = f'//*[@id="reviewSectionComments"]//*[@id="review-{i}"]/div[2]/div[1]/div[1]'
                review_element = review_section.find_element(By.XPATH, review_xpath)
                review_text = review_element.text.strip()
                
                if review_text:
                    reviews.append(review_text)
                
            except Exception as inner_e:
                if "no such element" in str(inner_e).lower():
                  break
                else:
                  print(f"Error retrieving review at index {i}: {inner_e}")
                  break
            i += 1
                
        return reviews
    except Exception as e:
        print(f"Error getting reviews: {str(e)}")
        return []

def label_reviews(rating_category, reviews):
    labels = {
        "9+ Hiếm Có": "positive",
        "8-9 Xuất Sắc": "positive",
        "7-8 Rất Tốt": "positive",
        "6-7 Tốt": "negative",
        "<6 Dưới Mức Mong Đợi": "negative"
    }
    
    labeled_reviews = []
    for review in reviews:
        if review:
            labeled_reviews.append({
                "review": review,
                "label": labels[rating_category]
            })
    return labeled_reviews

def get_hotel_name(driver):
    try:
        hotel_name_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="property-main-content"]/div[1]/div[2]/div[1]/h1'))
        )
        return hotel_name_element.text.strip()
    except Exception as e:
        print(f"Error getting hotel name: {str(e)}")
        return ""

def go_to_next_page(driver, current_page):
    try:
        next_page_xpath = f'//*[@id="reviews-panel-1"]/div[2]/div/span[2]/span[{current_page + 1}]'
        next_page_element = WebDriverWait(driver, 5).until( # Đổi từ 5 thành 10
            EC.element_to_be_clickable((By.XPATH, next_page_xpath))
        )
        scroll_to_element(driver, next_page_element)
        driver.execute_script("arguments[0].click();", next_page_element)
        time.sleep(2)  # Wait for the next page to load # Đổi từ 2 thành 5
        return True
    except (NoSuchElementException, TimeoutException):
        print("No next page found.")
        return False

def main():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-notifications')
    options.add_argument('--start-maximized')
    driver = webdriver.Chrome(options=options)

    try:
        # Gắn link của trang đánh giá khách sạn
        driver.get("https://www.agoda.com/homestay-john-cafe-and-beer/hotel/dalat-vn.html")
        time.sleep(3)

        # Lấy tên khách sạn
        hotel_name = get_hotel_name(driver)
        print(f"Tên khách sạn: {hotel_name}")

        review_section = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="reviewSectionComments"]'))
        )
        scroll_to_element(driver, review_section)
        driver.execute_script("arguments[0].click();", review_section)
        time.sleep(2)

        # Chọn ngôn ngữ tiếng Việt
        language_select_xpath = '//*[@id="reviewFilterSection"]/div[1]/div[3]/div/div/label/div[1]/span/select'
        language_select = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, language_select_xpath))
        )
        language_select = Select(language_select)
        language_select.select_by_visible_text("Tiếng Việt")
        time.sleep(2)

        rating_categories = {
            "9+ Hiếm Có": '//*[@id="reviews-panel-1"]/div[1]/div[1]/div/div[3]/div/div/div/div/div[1]/span',
            "8-9 Xuất Sắc": '//*[@id="reviews-panel-1"]/div[1]/div[1]/div/div[3]/div/div/div/div/div[2]/span',
            "7-8 Rất Tốt": '//*[@id="reviews-panel-1"]/div[1]/div[1]/div/div[3]/div/div/div/div/div[3]/span',
            "6-7 Tốt": '//*[@id="reviews-panel-1"]/div[1]/div[1]/div/div[3]/div/div/div/div/div[4]/span',
            "<6 Dưới Mức Mong Đợi": '//*[@id="reviews-panel-1"]/div[1]/div[1]/div/div[3]/div/div/div/div/div[5]/span'
        }

        data = []
        for category, xpath in rating_categories.items():
            rating = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, xpath))
            )
            scroll_to_element(driver, rating)
            driver.execute_script("arguments[0].click();", rating)
            time.sleep(2)

            current_page = 1
            while True:
                print(f"\n=== Reviews for {category} (Page {current_page}) ===")
                reviews = get_reviews_for_rating(driver)
                labeled_reviews = label_reviews(category, reviews)

                for i, review in enumerate(labeled_reviews, 1):
                    print(f"\nReview {i}:")
                    print(f"Text: {review['review']}")
                    print(f"Label: {review['label']}")

                for review in labeled_reviews:
                    data.append({
                        "Tên Khách Sạn": hotel_name,
                        "Review": review['review'],
                        "Label": review['label']
                    })

                if go_to_next_page(driver, current_page):
                    current_page += 1
                else:
                    break

            driver.execute_script("arguments[0].click();", rating)
            time.sleep(1)

        # Lưu dữ liệu vào file Excel
        df = pd.DataFrame(data)
        df.to_excel("C:/Users/Admin/Downloads/temp.xlsx", index=False)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

Tên khách sạn: Hostel-John Cafe and Beer

=== Reviews for 9+ Hiếm Có (Page 1) ===

Review 1:
Text: 10 điểm không có nhưng”
- Là 1 người sg chính hiệu nhưng lại muốn trở thành người đà lạt sau 2 ngày ở đây. - Phòng tập thể tuy hơi bất tiện nhưng lại sạch sẽ, ấm cúng, đầy đủ tiện nghi, có ngăn để đồ riêng, có ban công ở phòng rất chill. - Có khu sinh hoạt chung ở sân thượng, có rooftop để ngồi uống cf ngắm bình minh, hoàng hôn. - Đi du lịch 1 mình tiết kiệm thì còn nơi nào xứng đáng như vậy nữa 🫶
Label: positive

Review 2:
Text: CHỖ NÀY 10 ĐIỂM KHÔNG NÓI NHIỀU”
Mình thề với tầm giá tiền như thế này không thể ở một chỗ tốt hơn được. Phòng xịn, sạch sẽ, ấm áp, mình có căn nên đi du lịch ở mấy chỗ không phong thuỷ là mất ngủ, bóng đè, nhưng ngủ ở John giường như tẩm đá, ngủ ngon lắm. Chỗ ở trung tâm, mấy hàng ăn ngon toàn ở quanh phường này. Highly recommend!
Label: positive

Review 3:
Text: Dễ thương ấm cúng hợp cho chuyến xả hơi nhẹ nhàng”
Chỗ nghỉ nhỏ nhắn, ấm cúng, đồ dùng thì đầy đủ lắ

### **Xử lý các đoạn trước dấu "**

In [13]:
import pandas as pd
import re

# Đọc file Excel
input_file = r"C:/Users/Admin/Downloads/FinalReview.xlsx"
output_file = r"C:/Users/Admin/Downloads/Another.xlsx"

# Đọc dữ liệu vào DataFrame
df = pd.read_excel(input_file)

# Hàm xử lý cột Review để giữ lại nội dung sau dấu nháy kép (")
def clean_review(text):
    if isinstance(text, str) and '”' in text:
        # Tìm vị trí dấu nháy kép và lấy phần sau nó
        return re.split(r'”', text, 1)[-1].strip()
    return text

# Áp dụng hàm làm sạch cho cột Review
df['Review'] = df['Review'].apply(clean_review)

# Lưu DataFrame đã xử lý vào file Excel mới
df.to_excel(output_file, index=False)

print("File đã được xử lý và lưu thành công tại:", output_file)

File đã được xử lý và lưu thành công tại: C:/Users/Admin/Downloads/Another.xlsx
